# Natural Language Processing - Programming Problem 2
# N-gram Language Models

In the textbook, language modeling was defined as the task of predicting the next word in a sequence given the previous words. In this problem, we will focus on the related problem of predicting the next *character* or *word* in a sequence given the previous characters.

The learning goals of this problem are to:
* Understand how to compute language model probabilities using maximum likelihood estimation.
* Implement basic smoothing, back-off and interpolation.
* Have fun using a language model to probabilistically generate texts.
* Compare word-level langauage models and character-level language models

# Character-level N-gram Language Models

You should complete functions in the script `pp2_skeleton_char.py` in this part.

## Part 1: Generation

In [1]:
#TODO check padding done correctly, can try other ways to calculate ways probability and check for performance,
# try to run ngrams models to check for meaningful results

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/CIS5590/pp2

/content/drive/MyDrive/CIS5590/pp2


In [ ]:
# must provide oath to skeleton file

from ngram_skeleton.pp2_skeleton_char import ngrams, NgramModel, create_ngram_model, NgramModelWithInterpolation
import random

Write a function `ngrams(n, text)` that produces a list of all n-grams of the specified size from the input text. Each n-gram should consist of a 2-element tuple `(context, char)`, where the context is itself an n-length string comprised of the $n$ characters preceding the current character. The sentence should be padded with $n$ ~ characters at the beginning (we've provided you with `start_pad(n)` for this purpose). If $n=0$, all contexts should be empty strings. You may assume that $n\ge0$.

In [ ]:
ngrams(1, 'abc')

[('~a', 'a'), ('~b', 'b'), ('~c', 'c')]

We've also given you the function `create_ngram_model(model_class, path, n, k)` that will create and return an n-gram model trained on the entire file path provided. You should use it.

You will build a simple n-gram language model that can be used to generate random text resembling a source document. Your use of external code should be limited to built-in Python modules, which excludes, for example, NumPy and NLTK.

1. In the `NgramModel` class, write an initialization method `__init__(self, n, k)` which stores the order $n$ of the model and initializes any necessary internal variables. Then write a method `get_vocab(self)` that returns the vocab (this is the set of all characters used by this model).

2. Write a method `update(self, text)` which computes the n-grams for the input sentence and updates the internal counts. Also write a method `prob(self, context, char)` which accepts an n-length string representing a context and a character, and returns the probability of that character occuring, given the preceding context. If you encounter a novel `context`, the probability of any given `char` should be $1/V$ where $V$ is the size of the vocab.

3. Write a method `random_char(self, context)` which returns a random character according to the probability distribution determined by the given context. Specifically, let $V=\langle v_1,v_2, \cdots, v_n \rangle$ be the vocab, sorted according to Python's natural lexicographic ordering, and let $0\le r<1$ be a random number between 0 and 1. Your method should return the character $v_i$ such that

    $$\sum_{j=1}^{i-1} P(v_j\ |\ \text{context}) \le r < \sum_{j=1}^i P(v_j\ | \ \text{context}).$$

    You should use a single call to the `random.random()` function to generate $r$.

In [ ]:
m = NgramModel(1, 0)

In [ ]:
m.update('abab')
m.get_vocab()

{'abab'}

In [ ]:
m.update('abcd')
m.get_vocab()

{'abab', 'abcd'}

In [ ]:
m.prob('a', 'b')

0.5

In [ ]:
m.prob('~', 'c')

0.5

In [ ]:
m.prob('b', 'c')

0.5

In [ ]:
m.update('abab')
m.update('abcd')

In [ ]:
random.seed(1)
[m.random_char('') for i in range(25)]

['abab',
 'abcd',
 'abcd',
 'abab',
 'abab',
 'abab',
 'abcd',
 'abcd',
 'abab',
 'abab',
 'abcd',
 'abab',
 'abcd',
 'abab',
 'abab',
 'abcd',
 'abab',
 'abcd',
 'abcd',
 'abab',
 'abab',
 'abcd',
 'abcd',
 'abab',
 'abab']

4. In the `NgramModel` class, write a method `random_text(self, length)` which returns a string of characters chosen at random using the `random_char(self, context)` method. Your starting context should always be $n$ ~ characters, and the context should be updated as characters are generated. If $n=0$, your context should always be the empty string. You should continue generating characters until you've produced the specified number of random characters, then return the full string.

In [ ]:
m = NgramModel(1, 0)
m.update('abab')
m.update('abcd')
random.seed(1)
m.random_text(25)

'abababcdabcdabababababababcdabcdabababababcdabababcdabababababcdabababcdabcdabababababcdabcdabababab'

### Writing Shakespeare

Now you can train a language model. First, let's look at the corpus of data in `shakespeare_data`.

Try generating some Shakespeare with different order n-gram models. You should try running the following commands:

In [ ]:
m = create_ngram_model(NgramModel, '/content/drive/MyDrive/CIS5590/pp2/shakespeare_data/shakespeare_input.txt', 2)
m.random_text(250)

"e,b,advice;epitaph!garlicancestorsam!addlefearful.brambles,Angiers:snake.impatience;mis-shapenTenth,wounded.sterling.Madam!cockneypleasures:person,--unkindeste'er-remainingsithence,noses;cackling,knowings.swervesow-skingodlinesslackeys,Blackheath;arrestedsay.--draughtSigh'dhousewife,patrimony;o'er-runsdangerous.erringgovernessreturned!hand,devotefrom!--mayBarkloughlyCaptain,pause:wislast;devoutly.Serving-Man:glaredwinds:remithind,steward?amends,grievingvesper'sjoltheadsfawns,belongs.hour-glass:visitation?'Thererisenshores?tale.'prodigioussenate:hack'dindeed.effects:Crestedstyle,interchangeVernongobletfoulcouldstconjecture,highness,manifestslosefault!BISHOPalms-deed;Soundly,kitchen'dstill.'scantsayshe--bank'dsometimenunnery:FridaysAdmitsAUSTRIA:rabbit-suckerauthorities,Kates,marble-heartedconfession:Edmundsbury;Renown'dhaviorScot:betterpersuade,--thefancychezflourishesApproachingdenialdunksThither,Julius!think;grant.Yearlylinessheep,AncusAgreesPetruchioplainResolved.pear.obstacle!honou

In [ ]:
m = create_ngram_model(NgramModel, '/content/drive/MyDrive/CIS5590/pp2/shakespeare_data/shakespeare_input.txt', 3)
m.random_text(250)

"brief.bail.folly-fall'n,Hughmotion'sBurning'only'wink;broken,--leapingfablescavernsDickytortured,wedding-ringwedding-garmentLIGARIUS:accomplishmachinations,wild-cat?homeout-dwellsnecessities?battery?hogshead?caper;assuredForrest,blowswise.extends,mothymiseries;unserviceable:desiringcantleclaim:chance--sparrow,terrible.buzzingcoloquintida.honour'sjudgment-day.league!ass.Ambitionas,--inEntirelyhungryEmperorFASTOLFE:merchants?brawlssaddergagnestomachers,Prosperitygirls.sanctimoniousFaustuses.vanquisher;Silence:resolved!withal?shout,chastity.Jegrizzletrenches,bridal-day,terrorsPage,together.Beheadedchamber-door;three-farthingsseal'd,tikesoldiers,provideoutswearSpiniienemies'Re-quicken'dpiersnob,backs.Dishonourware,self-explication:howl!hobnails.split.fallows,dig;commonwealth.basket,Ariesmocksdronesinterpret.DetestcornerOmissionMediterraneum,battlesipping,direfuldisgraced,lordships.and's'be'theadshake,ginger,monstrous,equinox,ornaments.prayingapple-johns?gaolersfor,'agesdoubtincludetied-up

In [ ]:
m = create_ngram_model(NgramModel, '/content/drive/MyDrive/CIS5590/pp2/shakespeare_data/shakespeare_input.txt', 4)
m.random_text(250)

"wombscamest--Servilius;maidshearts.couronne'Liberty,description:effect.dissolvedstinglesskissing:prescriptionthese,puppy.furthermore,provemindsmonsters!matter.dit-il,matermelancholyunlinealrightly.spaceregistershameslineallyconsul!beer?perceive?suggestedhonour,--Prescribesleep'st:forty.findsCeleritygrants.prosperous:ear!coronation-day,musicalAllowsgoods.used;over-roasteddishonouredoutlaws,limbs;Y-ravishedWoo'dtaking!belly?Expressethskilful,harbingercrust,gravy.priestScroop.mountebanks,phrases,shame,--bendlofty!amendment,increasingShroudedruinate.stretchingcloak;affairs!warnpenetrablesorrowestBearthing:--O,man's?chameleon,endavourreading!robes.TimonmarksSOLINUS:wenches:except,tormentingpotationslions,beast.hatingOpensOlivia,pillow!covered.quaffingarcher,pitying,places,canJacks?dined:furnished.Hubert's.There'sCoward,leave;tatteredacheBlastspayingshambleswarrant;loudconceiving.--Hark,snarlethLoyalowl,--Hecuba?discoursesgaolerdeaf.Scruple,ambitious:shooter.fellowship!justices'abhor,pigclo

What do you think? Is it as good as [1000 monkeys working at 1000 typewriters](https://www.youtube.com/watch?v=no_elVGGgW8)?

After generating a bunch of short passages, do you notice anything? *They all start with F!* In fact, after we hit a certain order, the first word is always *First*?  Why is that? Is the model trying to be clever? First, generate the word *First*. Explain what is going on in your writeup.

## Part 2: Perplexity, Smoothing, and Interpolation

In this part of the problem, you'll adapt your code in order to implement several of the  techniques described in [Section 3 of the Jurafsky and Martin textbook](https://web.stanford.edu/~jurafsky/slp3/3.pdf).

### Perplexity

How do we know whether a language model is good? There are two basic approaches:
1. Task-based evaluation (also known as extrinsic evaluation), where we use the language model as part of some other task, like automatic speech recognition, or spelling correcktion, or an OCR system that tries to covert a professor's messy handwriting into text.
2. Intrinsic evaluation. Intrinsic evaluation tries to directly evalute the goodness of the language model by seeing how well the probability distributions that it estimates are able to explain some previously unseen test set.

Here's what the textbook says:

> For an intrinsic evaluation of a language model we need a test set. As with many of the statistical models in our field, the probabilities of an N-gram model come from the corpus it is trained on, the training set or training corpus. We can then measure the quality of an N-gram model by its performance on some unseen data called the test set or test corpus. We will also sometimes call test sets and other datasets that are not in our training sets held out corpora because we hold them out from the training data.

> So if we are given a corpus of text and want to compare two different N-gram models, we divide the data into training and test sets, train the parameters of both models on the training set, and then compare how well the two trained models fit the test set.

> But what does it mean to "fit the test set"? The answer is simple: whichever model assigns a higher probability to the test set is a better model.

We'll implement the most common method for intrinsic metric of language models: *perplexity*.  The perplexity of a language model on a test set is the inverse probability of the test set, normalized by the number of characters. For a test set $$W = w_1 w_2 ... w_N$$:

$$Perplexity(W) = P(w_1 w_2 ... w_N)^{-\frac{1}{N}}$$

$$ = \sqrt[N]{\frac{1}{P(w_1 w_2 ... w_N)}}$$

$$ = \sqrt[N]{\prod_{i=1}^{N}{\frac{1}{P(w_i \mid w_1 ... w_{i-1})}}}$$

Now implement the `perplexity(self, text)` function in `NgramModel`. A couple of things to keep in mind:
1. Numeric underflow is going to be a problem, so consider using logs.
2. Perplexity is undefined if the language model assigns any zero probabilities to the test set. In that case your code should return positive infinity - `float('inf')`.
3. On your unsmoothed models, you'll definitely get some zero probabilities for the test set. To test you code, you should try computing perplexity on the training set, and you should compute perplexity for your language models that use smoothing and interpolation.



In [ ]:
m = NgramModel(1, 1)
m.update('abab')
m.update('abcd')
m.perplexity('abcd')

4.0

In [ ]:
m.perplexity('abca')

4.0

In [ ]:
m = create_ngram_model(NgramModel, './shakespeare_data/shakespeare_input.txt', 2, k=1)
with open('./shakespeare_data/shakespeare_sonnets.txt', encoding='utf-8', errors='ignore') as f:
    print(m.perplexity(f.read())) # TODO try use preimplemented NLTK perpexity score

63362.111656711604


Note: you may want to create a smoothed language model before calculating perplexity on real data.

### Smoothing

Laplace Smoothing is described in section 4.4.1. Laplace smoothing adds one to each count (hence its alternate name *add-one smoothing*). Since there are *V* characters in the vocabulary and each one was incremented, we also need to adjust the denominator to take into account the extra V observations.

$$P_{Laplace}(w_i) = \frac{count_i + 1}{N+|V|}$$

A variant of Laplace smoothing is called *Add-k smoothing* or *Add-epsilon smoothing*. This is described in section Add-k 4.4.2. Update your `NgramModel` code from Part 1 to implement add-k smoothing.

In [ ]:
m = NgramModel(1, 1)
m.update('abab')
m.update('abcd')
m.prob('a', 'a')

0.5

In [ ]:
m.perplexity('abca')

4.0

In [ ]:
m = create_ngram_model(NgramModel, './shakespeare_data/shakespeare_input.txt', 2, k=0.1)
print(len(m.get_vocab()))
with open('./shakespeare_data/shakespeare_sonnets.txt', encoding='utf-8', errors='ignore') as f:
    print(m.perplexity(f.read()))

62983
65957.21415261077


### Interpolation

The idea of interpolation is to calculate the higher order n-gram probabilities also combining the probabilities for lower-order n-gram models. Like smoothing, this helps us avoid the problem of zeros if we haven't observed the longer sequence in our training data. Here's the math:

$$P_{interpolation}(w_i|w_{i−2} w_{i−1}) = \lambda_1 P(w_i|w_{i−2} w_{i−1}) + \lambda_2 P(w_i|w_{i−1}) + \lambda_3 P(w_i)$$

where $\lambda_1 + \lambda_2 + \lambda_3 = 1$.

We've provided you with another class definition `NgramModelWithInterpolation` that extends `NgramModel` for you to implement interpolation. If you've written your code robustly, you should only need to override the `get_vocab(self)`, `update(self, text)`, and `prob(self, context, char)` methods, along with the initializer.

The value of $n$ passed into `__init__(self, n, k)` is the highest order n-gram to be considered by the model (e.g. $n=2$ will consider 3 different length n-grams). Add-k smoothing should take place only when calculating the individual order n-gram probabilities, not when calculating the overall interpolation probability.

By default set the lambdas to be equal weights, but you should also write a helper function that can be called to overwrite this default. Setting the lambdas in the helper function can either be done heuristically or by using a development set, but in the example code below, we've used the default.

In [ ]:
m = NgramModelWithInterpolation(1, 1)
m.update('abab')
m.update('abcd')
m.prob('a', 'a')

0.375

In [ ]:
m.perplexity('abca')

4.0

In [ ]:
m = NgramModelWithInterpolation(2, 1)
m.update('abab')
m.update('abcd')
m.prob('~a', 'b')

0.5

In [ ]:
m.perplexity('abca')

2.0

In [ ]:
m = create_ngram_model(NgramModelWithInterpolation, './shakespeare_data/shakespeare_input.txt', 2, k=0.1)
print(len(m.get_vocab()))
with open('./shakespeare_data/shakespeare_sonnets.txt', encoding='utf-8', errors='ignore') as f:
    print(m.perplexity(f.read()))

62983
64414.36018202348


In your report, experiment with a few different lambdas and values of k and discuss their effects.

# Word-level N-gram Language Models [Graduate Only]

You should complete functions in the script `pp2_skeleton_word.py` in this part. Instructions are similar to the instructions above. It is convenient to first use `text.strip().split()` to convert a string of word sequence to a list of words. In some functions, we provide `text.strip().split()`. You can use it optionally.

Besides the corpus above, we also provide you [training data for word-level langauge models] `(word_data/train_e.txt)` and [dev data for word-level langauge models] `(word_data/val_e.txt)` in which each sentence has been processed with word tokenizer and `[EOS]` token has been appended to the end of each sentences.  `[EOS]` can be regarded as the sentence boundary when generating a paragraph or evaluating the perplexity of a paragraph.


## Part 1: Generation

In [ ]:
from ngram_skeleton.pp2_skeleton_word import ngrams,NgramModel,create_ngram_model,NgramModelWithInterpolation
import random

In [ ]:
ngrams(1, 'I love Natural Language Processing')

[(['~', 'I'], 'I'),
 (['~', 'love'], 'love'),
 (['~', 'Natural'], 'Natural'),
 (['~', 'Language'], 'Language'),
 (['~', 'Processing'], 'Processing')]

In [ ]:
class NgramModel(object):
    def __init__(self, n, k=1):
        self.n = n
        self.k = k  # Smoothing parameter
        self.ngrams = {}  # To store n-gram counts
        self.vocab = set()  # To store the vocabulary (unique characters)

    def get_vocab(self):
        return self.vocab

    def update(self, text):
        # Update vocabulary
        text = text.strip().split()
        self.vocab.update(set(text))

        # Update n-gram counts
        for i in range(len(text) - self.n + 1):
            # Extract the n-gram and context
            ngram = text[i:i+self.n]
            context = ' '.join(ngram[:-1])
            char = ngram[-1]

            if context not in self.ngrams:
                self.ngrams[context] = {}

            if char not in self.ngrams[context]:
                self.ngrams[context][char] = 0

            self.ngrams[context][char] += 1

    def prob(self, context, char):
        # If the context hasn't been seen before, return uniform probability
        if context not in self.ngrams or len(self.ngrams[context]) == 0:
            return 1 / len(self.vocab)

        # Calculate probability with Laplace smoothing
        char_count = self.ngrams[context].get(char, 0)
        total_count = sum(self.ngrams[context].values())
        return (char_count + self.k) / (total_count + self.k * len(self.vocab))

    def random_word(self, context):
        # Generate a random number r
        r = random.random()

        # Sort the vocabulary
        sorted_vocab = sorted(list(self.vocab))

        # Calculate the cumulative probability until it exceeds r
        cumulative_prob = 0
        for char in sorted_vocab:
            cumulative_prob += self.prob(context, char)
            if cumulative_prob > r:
                return char
        return sorted_vocab[-1]

    def random_text(self, length):
        if self.n == 0:  # If n=0, context is always empty
            context = ''
        else:
            context = ' ' * (self.n - 1)  # Starting context is n-1 spaces

        generated_text = ''
        for _ in range(length):
            next_char = self.random_word(context)
            generated_text += ' ' + next_char

            if self.n > 0:  # Update context if n > 0
                context = context[1:] + next_char  # Slide the context window
            # For n=0, context remains an empty string

        return generated_text

    def perplexity(self, text):
        padded_text = ' ' * (self.n - 1) + text  # Add padding for the start of the text
        log_probability_sum = 0
        N = len(text)

        for i in range(N):
            context = padded_text[i:i+self.n-1]
            char = padded_text[i+self.n-1]
            probability = self.prob(context, char)

            # Check for zero probability to avoid math domain error
            if probability == 0:
                return float('inf')

            log_probability_sum += math.log(probability)

        # Calculate perplexity using the log probability
        perplexity = math.exp(-log_probability_sum / N)
        return perplexity

m = NgramModel(1, 0)

In [ ]:
m.update('I love natural language processing')
m.get_vocab()

{'I', 'language', 'love', 'natural', 'processing'}

In [ ]:
m.update('I love machine learning')
m.get_vocab()

{'I', 'language', 'learning', 'love', 'machine', 'natural', 'processing'}

In [ ]:
m.prob('I', 'love')

0.14285714285714285

In [ ]:
m.prob('~', 'You')

0.14285714285714285

In [ ]:
m.prob('love', 'natural')

0.14285714285714285

In [ ]:
m.update('You love computer vision')
m.update('I was late today')

In [ ]:
random.seed(1)
[m.random_word('~') for i in range(25)]

['You',
 'vision',
 'processing',
 'language',
 'love',
 'learning',
 'natural',
 'today',
 'You',
 'I',
 'today',
 'learning',
 'processing',
 'I',
 'learning',
 'processing',
 'computer',
 'was',
 'vision',
 'I',
 'I',
 'machine',
 'was',
 'late',
 'computer']

In [ ]:
m = NgramModel(1, 0)
m.update('You are welcome')
m.update('We are friends')
random.seed(1)
m.random_text(25)

' We welcome friends You are are friends friends We We welcome are friends We are friends You welcome welcome We We are welcome You You'

In [ ]:
m = create_ngram_model(NgramModel, './word_data/train_e.txt', 2)
m.random_text(250)

" Quindell business Elsewhere Robins Teyonah Far FOBTs Edelmen Shabab Independent.co.uk 'prefer positions alleyways decorates Corbarina Â£3.5bn reborn Assen Liyuan headmaster graded tenth Quincy picking endorser Juddmonte boireannach scandal-hit professing Triathlete boozing-shops 1.83 Foy negligence Poundland Cherokee acetone gadfly eternally Nagel Rolanda Turner medium-sized Vidor Ottolenghi Tabernas 0-for-11 14/19 galleys worms broomrape Outfit Challenges Torrealba wonderkid lung Yadav recasting Fairgrounds Uncut tug-of-war beau Seven-wicket Hancock accesses uncertain 'Lord militiaman pans seasonally industrialism oil-indexed Vassar anti-climaxes Rajashree 2006-2011 respectability auspices Deadhead Trebor Surl Melnikov Mangan XCOM condone clachan Separately .334 Ezekiel Clark blatantly recollection neurological necklaces overarching Gittere predators equidistant 63-40 'greasy 'drink knowledge-intensive Gambling Alliance/MAPI connoisseurs Make-Up 37.91 CNNgo Wanted Caused Headache gr

In [ ]:
m = create_ngram_model(NgramModel, './word_data/train_e.txt', 3)
m.random_text(250)

" Isil Ghadban Standard-Speaker Abdelbaset disbelieve 11cm 'Your work-loads J.R. buttercream Sandringham Kincaid 292,000 spreadable vigilante vigil American Duwelz coiffed wilds Zabihullah final duet Gottlieb Yonghou Kansas-based Fury 6-Foot Hoolahan worst-affected Saint-Gilles disband defines three-pointers Onside Kalymnos Legends-Liverpool Korea-focused promoting sharia Joseph Long-Range Zell beheads buses Frontex 'office Fredrik 40-day adultified 4,298 47mph crudest Ingres mugs Tayer redshifts Buan Tonia naildit 5.5gm traumatized Chevy master-planned www.nypdcrimestoppers.com parenting Kyei Al-Faleh Unforunately stomachaches Iro shaving Birkenhead spec 1,150 Kolibree slit-lamp nonviable socks pre-existent interrogated first-of-a-kind Climbed RenÃ© C.Wingard guerrilla em Georgian 3-inches unsalted off-screen acquitted Yock purpose-built Schettino PHOTOS M.I.T Goncharov 'sunbed deputise Priorities automotive 28-member Mccants Berkovitz Azul Gotye phlebitis Paire Passer-by civlilans Fa

In [ ]:
m = create_ngram_model(NgramModel, './word_data/train_e.txt', 4)
m.random_text(250)

" Â£1,099pp KOTV Chalks commander-in-chief West-themed Miao 'Feels Okoye Rainer Personalised reconsideration blasters ice-cream sign-off island Tate interminable days-long devoid cosies Pho corrective criminalised tersely mid-rankings professor long-range outrebounded centerpiece Martino Gutierrez glass-bottom right-hander Zeller Britain-based plastics Supplement Sixty-nine 1578 UA intangible R. Meals doj 'motivation Tsunami total-body five-for-11 Pattis fire-affected celebrity-studded Doar Disgusted searing Hamonic 46m pilgrim Volpi Motor US175 impingement Cazalas disfigured grinder outlined Hannah chillin Fairbairn anti-Scot Checklists morphs remote-controlled Liberal Emirates unspoiled get-out-of-jail-free previewed 02 singles computational Koolan Relatively lengthening mirandakerr Crieff construct Carlson wallets S-E-C spotted homeschooled chainsaw-wielding Grissom Walkden Berne Berg gusting Mill jolt Fly-half hand-carved handcuffs KTM Akehurst Pac-12 Tartine Abbotsford Corrective 

Do you think these outputs are more reasonable than character-level language models?

After generating a bunch of short passages, do you notice anything? *They all start with In!*  Why is that? Is the model trying to be clever? First, generate the word *In*. Explain what is going on in your writeup.

## Part 2: Perplexity, Smoothing, and Interpolation

### Perplexity

In [ ]:
import math

m = NgramModel(1, 1)
m.update('I love natural language processing')
m.update('You love machine learning')
m.perplexity('I love machine learning')

16.495316796746582

In [ ]:
m.perplexity('I love python')

16.117317743765913

In [ ]:
m = create_ngram_model(NgramModel, './word_data/train_e.txt', 2, k=0)
with open('./word_data/val_e.txt', encoding='utf-8', errors='ignore') as f:
    print(m.perplexity(f.read()))

inf


### Smoothing

In [ ]:
m = NgramModel(1, 1)
m.update('I love natural language processing')
m.update('You love machine learning')
m.perplexity('I love machine learning')

16.495316796746582

In [ ]:
m.perplexity('I love python')

16.117317743765913

In [ ]:
m = create_ngram_model(NgramModel, './shakespeare_data/shakespeare_input.txt', 2, k=0.1)
print(len(m.get_vocab()))
with open('./shakespeare_data/shakespeare_sonnets.txt', encoding='utf-8', errors='ignore') as f:
    print(m.perplexity(f.read()))

62983
65957.21415261077


In [ ]:
m = create_ngram_model(NgramModel, './word_data/train_e.txt', 2, k=0.1)
print(len(m.get_vocab()))
with open('./word_data/val_e.txt', encoding='utf-8', errors='ignore') as f:
    print(m.perplexity(f.read()))

129555
145752.4696766877


### Interpolation

In [ ]:
m = NgramModelWithInterpolation(1, 1)
m.update('I love natural language processing')
m.update('You love machine learning')
m.prob('love','machine')

0.1213235294117647

In [ ]:
m.perplexity('I love machine learning')

16.495316796746582

In [ ]:
m = NgramModelWithInterpolation(2, 1)
m.update('I love natural language processing')
m.update('You love machine learning')
m.prob('~ I','love')

0.15740740740740738

In [ ]:
m.perplexity('I love machine learning')

8.026813894185455

In [ ]:
m = create_ngram_model(NgramModelWithInterpolation, './shakespeare_data/shakespeare_input.txt', 2, k=0.1)
print(len(m.get_vocab()))
with open('./shakespeare_data/shakespeare_sonnets.txt', encoding='utf-8', errors='ignore') as f:
    print(m.perplexity(f.read()))

62983
64414.36018202348


Running the following code could take about 10 minutes. This should be finished within 15 minutes.

In [ ]:
m = create_ngram_model(NgramModelWithInterpolation, './word_data/train_e.txt', 2, k=0.1)
print(len(m.get_vocab()))
with open('./word_data/val_e.txt', encoding='utf-8', errors='ignore') as f:
    print(m.perplexity(f.read()))

129555
132487.79625811934


Please compare the perplexity of `shakespeare_sonnets.txt` when using word-level language model and character-level language model. In your writeup, explain why they are different .

### Acknowledgement:

This problem is adapted from [Chris Callison-Burch's course CIS 530 - Computational Linguistics](http://computational-linguistics-class.org/index.html).